In [95]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import numpy as np

In [96]:

df = pd.read_excel ('IE5400_HW04-location selection.xlsx',header=1)
df.head()

,Unnamed: 0,Hospital,x,y,# Cases,Unnamed: 5,Warehouse,x.1,y.1,Capacity,Setup Cost
0,NaN,H1,12,12,230,NaN,w1,4.0,5.0,3300.0,8000.0
1,NaN,H2,9,4,280,NaN,w2,8.0,3.0,3000.0,6000.0
2,NaN,H3,6,9,145,NaN,w3,7.0,7.0,3500.0,9000.0
3,NaN,H4,3,9,112,NaN,w4,6.0,6.0,3600.0,9000.0
4,NaN,H5,8,2,235,NaN,w5,10.0,8.0,4000.0,12000.0


In [97]:
input_hospital_data = df.iloc[:, 1:5]
input_hospital_data

,Hospital,x,y,# Cases
0,H1,12,12,230
1,H2,9,4,280
2,H3,6,9,145
3,H4,3,9,112
4,H5,8,2,235
5,H6,4,1,405
6,H7,6,4,290
7,H8,5,7,370
8,H9,1,8,189
9,H10,4,6,405


In [98]:
input_warehouse_data = df.iloc[0:5, 6:11]
input_warehouse_data

,Warehouse,x.1,y.1,Capacity,Setup Cost
0,w1,4.0,5.0,3300.0,8000.0
1,w2,8.0,3.0,3000.0,6000.0
2,w3,7.0,7.0,3500.0,9000.0
3,w4,6.0,6.0,3600.0,9000.0
4,w5,10.0,8.0,4000.0,12000.0


In [99]:
# get the coordinates of hopsitals and warehouses
coordiate_hospital = input_hospital_data.iloc[:, 1:3].values
coordiate_warehouse = input_warehouse_data.iloc[:,1:3].values

In [100]:
# get the demand of hospital, capacity and setup cost of warehouses
demand=input_hospital_data.iloc[:,-1].astype(int).values
supply=input_warehouse_data.iloc[:,3].values
setup_cost=input_warehouse_data.iloc[:,4].values

In [109]:
# calcaute the distance between hospital and warehouses
distance=np.zeros((20,5))
for i in range(20):
    for j in range(5):
        distance[i,j]=((coordiate_hospital[i][0]-coordiate_warehouse[j][0])**2 + 
                       (coordiate_hospital[i][1]-coordiate_warehouse[j][1])**2)**0.5
distance

array([[10.63014581,  9.8488578 ,  7.07106781,  8.48528137,  4.47213595],
       [ 5.09901951,  1.41421356,  3.60555128,  3.60555128,  4.12310563],
       [ 4.47213595,  6.32455532,  2.23606798,  3.        ,  4.12310563],
       [ 4.12310563,  7.81024968,  4.47213595,  4.24264069,  7.07106781],
       [ 5.        ,  1.        ,  5.09901951,  4.47213595,  6.32455532],
       [ 4.        ,  4.47213595,  6.70820393,  5.38516481,  9.21954446],
       [ 2.23606798,  2.23606798,  3.16227766,  2.        ,  5.65685425],
       [ 2.23606798,  5.        ,  2.        ,  1.41421356,  5.09901951],
       [ 4.24264069,  8.60232527,  6.08276253,  5.38516481,  9.        ],
       [ 1.        ,  5.        ,  3.16227766,  2.        ,  6.32455532],
       [ 6.        ,  2.82842712,  3.60555128,  4.12310563,  3.        ],
       [ 8.24621125,  4.        ,  6.40312424,  6.70820393,  5.38516481],
       [ 2.        ,  4.        ,  5.        ,  3.60555128,  7.81024968],
       [ 8.94427191, 10.        ,  6.0

In [104]:
# question 1

I=range(20)
J=range(5)

m = gp.Model()

# x_ij whether hospital i is supplied by warehouse j
x = m.addVars(20, 5, vtype=GRB.BINARY, name='x')

# minimize the total travel distance
m.setObjective(sum(distance[i][j]*x[i,j] for i in I for j in J), GRB.MINIMIZE)

# each hosptial has to be supplied by one warehouse
m.addConstrs(sum(x[i,j] for j in J) == 1 for i in I)


m.optimize()

for v in m.getVars():
    print('%s = %g' % (v.varName, v.x))

print('Obj: %g' % m.objVal)

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 20 rows, 100 columns and 100 nonzeros
Model fingerprint: 0xccbd31e9
Variable types: 0 continuous, 100 integer (100 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 101.9671723
Presolve removed 20 rows and 100 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.04 seconds
Thread count was 1 (of 4 available processors)

Solution count 2: 62.1487 

Optimal solution found (tolerance 1.00e-04)
Best objective 6.214869314340e+01, best bound 6.214869314340e+01, gap 0.0000%
x[0,0] = 0
x[0,1] = 0
x[0,2] = 0
x[0,3] = 0
x[0,4] = 1
x[1,0] = 0
x[1,1] = 1
x[1,2] = 0
x[1,3] = 0
x[1,4] = 0
x[2,0] = 0
x[2,1] = 0
x[2,2] = 1
x[2,3] = 0
x[2,4]

In [87]:
# question 2

In [106]:

I=range(20)
J=range(5)


m = gp.Model()

# x_ij whether hospital i is supplied by warehouse j
x = m.addVars(20, 5, vtype=GRB.BINARY, name='x')
# y_j whether to setup warehouse j
y = m.addVars(5, vtype=GRB.BINARY, name='y')

# minimize total distance
m.setObjective(sum(distance[i][j]*x[i,j] for i in I for j in J), GRB.MINIMIZE)

# only install two warehouses
m.addConstr(sum(y[j] for j in J)==2)

# each hosptial has to be supplied by one warehouse
m.addConstrs(sum(x[i,j] for j in J) == 1 for i in I)

# if warehouse j is not setup, warehouse j cannot supply any hospital
m.addConstrs(x[i,j] <= y[j] for i in I for j in J)

# the total demand has to be less than the warehouse capacity
m.addConstrs(sum(demand[i]*x[i,j] for i in I) <= supply[j] for j in J)


m.optimize()

for v in m.getVars():
    print('%s = %g' % (v.varName, v.x))

print('Obj: %g' % m.objVal)

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 126 rows, 105 columns and 405 nonzeros
Model fingerprint: 0xa324fbdb
Variable types: 0 continuous, 105 integer (105 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+02]
  Objective range  [1e+00, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+03]
Found heuristic solution: objective 98.4366223
Presolve time: 0.00s
Presolved: 126 rows, 105 columns, 405 nonzeros
Variable types: 0 continuous, 105 integer (105 binary)

Root relaxation: objective 7.786673e+01, 76 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   77.86673    0   47   98.43662   77.86673  20.9%     -    0s
H    0     0                      83.6435512   77.86673  6.91%     -    0s
H    0     0         

In [ ]:
# question 3


In [110]:

I=range(20)
J=range(5)


m = gp.Model()

x = m.addVars(20, 5, vtype=GRB.BINARY, name='x')
y = m.addVars(5, vtype=GRB.BINARY, name='y')

# minimize total distance and setup cost
m.setObjective(sum(distance[i][j]*x[i,j] for i in I for j in J) + sum(setup_cost[j]*y[j] for j in J), GRB.MINIMIZE)

# each hosptial has to be supplied by one warehouse
m.addConstrs(sum(x[i,j] for j in J) == 1 for i in I)

# if warehouse j is not setup, warehouse j cannot supply any hospital
m.addConstrs(x[i,j] <= y[j] for i in I for j in J)


m.optimize()

for v in m.getVars():
    print('%s = %g' % (v.varName, v.x))

print('Obj: %g' % m.objVal)

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 120 rows, 105 columns and 300 nonzeros
Model fingerprint: 0xbecb7e8c
Variable types: 0 continuous, 105 integer (105 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 44115.464828
Presolve time: 0.00s
Presolved: 120 rows, 105 columns, 300 nonzeros
Variable types: 0 continuous, 105 integer (105 binary)

Root relaxation: objective 6.113285e+03, 101 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    6113.2845698 6113.28457  0.00%     -    0s

Explored 0 nodes (101 simplex iterations) in 0.02 seconds
Thread count was 4 (of 4 available